In [1]:
## to run in colab

!git clone https://github.com/sergeychuvakin/advanced_nlp_course.git
!mv advanced_nlp_course/LM/*.py ./
!mv advanced_nlp_course/LM/*.json ./ ## 
!pip install loguru pydantic tokenizers
!pip install nltk==3.6.2
import nltk
nltk.download('stopwords')
nltk.download('punkt')

Cloning into 'advanced_nlp_course'...
remote: Enumerating objects: 98, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 98 (delta 45), reused 80 (delta 30), pack-reused 0
Unpacking objects: 100% (98/98), done.
     |████████████████████████████████| 57 kB 2.8 MB/s 
     |████████████████████████████████| 10.9 MB 9.5 MB/s 
     |████████████████████████████████| 6.8 MB 45.3 MB/s 
     |████████████████████████████████| 1.5 MB 5.3 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
!pip freeze | egrep "pydantic|torch|loguru|tokenizers|requests|nltk|tqdm"

loguru==0.5.3
nltk==3.6.2
pydantic==1.9.0
requests==2.23.0
requests-oauthlib==1.3.0
tokenizers==0.11.4
torch @ https://download.pytorch.org/whl/cu111/torch-1.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
torchaudio @ https://download.pytorch.org/whl/cu111/torchaudio-0.10.0%2Bcu111-cp37-cp37m-linux_x86_64.whl
torchsummary==1.5.1
torchtext==0.11.0
torchvision @ https://download.pytorch.org/whl/cu111/torchvision-0.11.1%2Bcu111-cp37-cp37m-linux_x86_64.whl
tqdm==4.62.3


In [2]:
%reload_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
from loguru import logger
import sys
import json
from tqdm import tqdm


from dependencies import corpus, tokenizer
from config import Config, LanguageModelConfig
from processing_utils import (
    clean_text, 
    split_on_sequences, 
    create_ngrams, 
    create_ngrams1,
    create_to_x_and_y, 
    word2int,
    create_vocab,
    save_artifacts,
    remove_stopwords, 
    remove_speakers
)
from model import LM_LSTM
from datahandler import LMDataset
from train_utils import train_model

config = Config()

logger.remove()
logger.add(sys.stderr, level="WARNING")

1

In [3]:
corpus = remove_speakers(corpus)
corpus = clean_text(corpus)
corpus = remove_stopwords(corpus)
corpus = split_on_sequences(corpus)

# tcorpus = tokenizer.encode_batch(corpus, add_special_tokens=False)

# ## create n-grams for each doc
# sq = create_ngrams(tcorpus, config.N_GRAM) 
 
# ## shift corpus to create x and y 
# x, y =  create_to_x_and_y(sq)

# id_token, token_id = create_vocab(tokenizer)
# vocab_size = len(token_id)

from itertools import chain

tcorpus = [tokenizer(sent) for sent in tqdm(corpus)]

id_token = dict(enumerate(set(chain.from_iterable(tcorpus))))

## add special symbols 
id_token[len(id_token)] = Config.TOKEN_UNKNOWN ## add UNK
id_token[len(id_token)] = Config.TOKEN_PADDING ## add PAD

token_id = {value:key for key, value in id_token.items()}
vocab_size = len(token_id)

sq = create_ngrams1(tcorpus, config.N_GRAM, token_id)


100%|██████████| 51324/51324 [00:00<00:00, 264418.59it/s]


In [37]:
len(id_token)

48560

In [38]:
## shift corpus to create x and y 
x, y =  create_to_x_and_y(sq)

## split data
tradeoff_index = int(len(x) * config.TRAIN_PROPORTION)

x_train = x[:tradeoff_index]
x_test = x[tradeoff_index:]

y_train = y[:tradeoff_index]
y_test = y[tradeoff_index:]

logger.warning(
    f"""
    Output shapes: 
        x_train: {len(x_train)}, 
        x_test: {len(x_test)}, 
        y_train: {len(y_train)}, 
        y_test: {len(y_test)}
    """
              )

## load to dataset and dataloader
train_ds = LMDataset(x_train, y_train)
test_ds = LMDataset(x_test, y_test)

train_dl = DataLoader(train_ds, batch_size=config.BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=config.BATCH_SIZE, shuffle=False)

# model and model config
model_config = LanguageModelConfig(vocab_size=vocab_size, emb_size=300)
model = LM_LSTM(**model_config.dict(), logger=logger)

## save artifacts
save_artifacts(
    (model_config.dict(), config.SAVE_MODEL_CONFIG),
    (token_id, config.SAVE_TOKEN_ID),
    (id_token, config.SAVE_ID_TOKEN)
)

2022-01-22 11:56:58.278 | WARNING  | __main__:<module>:20 - 
    Output shapes: 
        x_train: 206670, 
        x_test: 33645, 
        y_train: 206670, 
        y_test: 33645
    


In [39]:
optimizer = torch.optim.Adam(model.parameters(), lr=model_config.lr, weight_decay=1e-5)
loss_func = torch.nn.CrossEntropyLoss()


# train model 
tmodel = train_model(model,
                     train_dl,
                     optimizer=optimizer,
                     loss_func=loss_func,
                     token_id=token_id,
                     epochs=1, 
                     clip=1)

torch.save(tmodel.state_dict(), config.SAVE_MODEL_FNAME)

  1%|          | 2/207 [00:00<01:08,  2.98it/s]

Perplexity loss: 48533.08203125



  1%|▏         | 3/207 [00:00<01:05,  3.12it/s]

Perplexity loss: 48358.8984375



  2%|▏         | 4/207 [00:01<01:03,  3.19it/s]

...

Perplexity loss: 47838.640625



 99%|█████████▉| 205/207 [01:01<00:00,  3.33it/s]

Perplexity loss: 47982.33984375



100%|██████████| 1/1 [01:01<00:00, 61.55s/it]

Perplexity loss: 47966.37109375


In [42]:
from validation_utils import val_metrics

logger.warning(
    """
    Cross-Entropy: %f
    Perpelxity: %f
    """ % 
    val_metrics(model, test_dl, token_id, loss_func)
)

2022-01-22 11:58:16.195 | WARNING  | __main__:<module>:8 - 
    Cross-Entropy: 10.779922
    Perpelxity: 48046.355469
    


### Inference

In [43]:

from inference import (
    model, 
    token_id, 
    id_token, 
    model, 
    predict_one_word, 
    predict_sample
)

from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, level="WARNING")

9

In [44]:
id_token[str(33274)]

'piercecorslet'

In [45]:
predict_one_word(
    "one of the jns sjdnvjs aldsv ",  
    model, 
    token_id, 
    id_token,
    random=False
)

'thou'

In [46]:
len(token_id)

48560

In [49]:
predict_sample(
    "how apply",  
    model, 
    token_id, 
    id_token,
    length_of_sample=10,
    random=False
)

'thou thou thou thou thou thou thou thou thou thou'